In [ ]:
import numpy as np
from SALib.sample import morris
from SALib.analyze import morris as morris_analyze
import os
from simulation import Simulation
from grid import Grid
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
def gini_coefficient(wealths):
    n = len(wealths)
    sorted_wealths = sorted(wealths)
    cumulative_wealths = [sum(sorted_wealths[:i+1]) for i in range(n)]
    numerator = sum((i + 1) * sorted_wealths[i] for i in range(n)) - (n + 1) * cumulative_wealths[-1] / 2
    denominator = n * cumulative_wealths[-1]
    return numerator / denominator

# Ensure the directory exists for saving the results
save_dir = "morris_results"
os.makedirs(save_dir, exist_ok=True)

baseline_params = {
    'num_agents': 50,
    'n_timesteps': 300,
    'num_resources': 500,
    'lifetime_mean': 80,
    'lifetime_std': 10,
    'resource_spawn_rate': 0.5,
    'order_expiry_time': 5,
    'tax_period': 3,
    'income_per_timestep': 3
}

problem = {
    'num_vars': 9,
    'names': ['num_agents', 'n_timesteps', 'num_resources', 'lifetime_mean', 'lifetime_std', 
              'resource_spawn_rate', 'order_expiry_time', 'tax_period', 'income_per_timestep'],
    'bounds': [[10, 60], [100, 500], [100, 600], [50, 110], [5, 20], [0.1, 1.0], [1, 10], [1, 10], [1, 10]]
}

param_values = morris.sample(problem, N=100, num_levels=4, optimal_trajectories=None)

results = []
for i in range(len(param_values)):
    sim_params = baseline_params.copy()
    for j, param in enumerate(problem['names']):
        # Cast float values to int where necessary
        if param in ['num_agents', 'n_timesteps', 'num_resources', 'lifetime_mean', 'lifetime_std', 'order_expiry_time', 'tax_period', 'income_per_timestep']:
            sim_params[param] = int(param_values[i, j])
        else:
            sim_params[param] = param_values[i, j]
    sim = Simulation(**sim_params, grid=Grid(width=60, height=60, house_cost=(2, 2)), show_time=True, dynamic_tax=True, dynamic_market=True)
    sim.run()
    results.append(sim.data)
    sim.save_results(f'morris_results/simulation_{i}_analysis.csv')

In [ ]:
gini_coefficients = [gini_coefficient(pd.DataFrame(run)['wealth']) for run in results]
gini_coefficients = np.array(gini_coefficients)

# Perform Morris sensitivity analysis
Si = morris_analyze.analyze(problem, param_values, gini_coefficients, conf_level=0.95, print_to_console=True)

In [ ]:
# Plotting results
plt.figure(figsize=(10, 6))
plt.bar(problem['names'], Si['mu_star'], yerr=Si['mu_star_conf'], capsize=5)
plt.title('Morris Method - Mean of Elementary Effects Based on Gini Coefficient (\u03BC*)')
plt.ylabel('\u03BC*')
plt.xlabel('Parameters')
plt.xticks(rotation=45, ha='right')  
plt.tight_layout()  
plt.savefig('morris_results/morris_sensitivity.png')
plt.show()